<a href="https://colab.research.google.com/github/Location-Artistry/ColabNotebooks/blob/master/waterDataExport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Working Water Data Pandas AWQMS Export**
Updated August 12th - Working with pandas dataframe exporting to excel   
Working function to run download and CSV formatting in pandas   
**8-14 Completed: Download & Export Functions, QuickMap Display**
**8-25 delMultiple, updated Kzoo Dashboard Status**   
**8-25 This Notebook has most current Python Functions!!!**

# Install and Import Libraries
**RUN FIRST**

In [ ]:
!pip install xlsxwriter
!pip install xlrd
!pip install openpyxl
!pip install arcgis
#!pip install pdfkit
#import pdfkit
import pandas as pd
from pathlib import Path
import sys
import os
import datetime as dt
import time as tm
import math
from arcgis.gis import GIS, Item
from arcgis.env import active_gis
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from IPython.display import display
import getpass
from pathlib import Path
from zipfile import ZipFile
from openpyxl import load_workbook

# Python Functions from my ArcGIS Python Colab Notebook  
**MUST RUN THIS CELL AFTER INSTALL AND IMPORT THEN - userLogin() -**


In [ ]:
# Collection of all ArcGIS Python API Helper Functions
# user login functions, ask if user would like additional logins
def userLogin():
    userID = input(f'ArcGIS Online USER ID: ')
    passWord = getpass.getpass('PASSWORD: ')
    try:
        global gis
        gis = GIS("https://www.arcgis.com", userID, passWord)
        print(f'SUCCESS - CONNECTED TO: {gis.users.me.username} ACCOUNT as <gis>')
        print(gis)
        addUsers = input(f'Additional User Login(YES/NO)? ')
        if addUsers.upper() == 'YES':
            additionalUserLogin()
        else:
            print(f'YOU MAY NOW PROCEED...')
    except:
        print(f'ERROR DID NOT CONNECT TO: {userID}')

def additionalUserLogin():
    userID = input(f'ArcGIS Online USER ID: ')
    passWord = getpass.getpass('PASSWORD: ')
    try:
        global gis2
        gis2 = GIS("https://www.arcgis.com", userID, passWord)
        print(f'SUCCESS - CONNECTED TO: {gis2.users.me.username} ACCOUNT as <gis2>')
        print(gis2)
    except:
        print(f'ERROR DID NOT CONNECT TO: {userID}')

# get list of all owner AGOL items, print list with title, id, type, and categories
def getUserContent(gisInfo):
    try:
        my_content = gisInfo.content.search(query="owner:" + gisInfo.users.me.username, item_type="", max_items=200)
        for x in my_content:
            strMod = str(x.modified)
            stampInt = int(strMod[0:10])
            print(f'{x.title} - {x.id} - {x.type} - {x.categories} - {dt.datetime.fromtimestamp(stampInt)}')
    except:
        print('ERROR could not get user content')

# Clone item using id of item passed to function
def cloneItem(gisInfo, gisInfo2, cloneID):    
    try:
        itemToClone = gisInfo.content.get(cloneID)
        print('Cloning:' + itemToClone.title + ' - ' + itemToClone.id + ' -',itemToClone.type)
        clonedItem = gisInfo2.content.clone_items(items=[itemToClone])
        print(f'Cloned Item: {clonedItem[0]}')
        #return clonedItem
    except:
        print('ERROR Could Not Clone')

# updated searchByKeywords, returns LIST of items 8-14-2020
def searchByKeywords(gisInfo, searchKeywords):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type='', max_items=50)
        x = 0
        for z in searchContent:
          strMod = str(z.modified)
          stampInt = int(strMod[0:10])
          print(f'{x} - {z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
          x += 1
        return searchContent
    except:
        print('ERROR Search not Successful')

# find item by keywords and display visual card
def searchByKeyViz(gisInfo, searchKeywords):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type='', max_items=50)
        for z in searchContent:
            print(f'title: {z.title} - itemID: {z.id} - type: {z.type}')
            display(z)
    except:
        print('ERROR Search not Successful')
        
# return all keys and values for item when passed itemID string
def getItemKeysValues(gisInfo, idString):
    try:
        getFeature = gisInfo.content.get(idString)
        for key, value in getFeature.items():
            print(key,': ', value)
    except:
        print('ERROR GET Keys/Values not Successful')
        
# takes itemID and gets and returns layerObject if exist, otherwise 'no layers found'
def getLayers(gisInfo, idString):
    getFeature = gisInfo.content.get(idString)
    try:
        featureLayers = getFeature.layers
        z = 0 
        for x in featureLayers:
            print(f'Layer {z}: {x}')
            z += 1
    except:
        print('no layers found')
    return featureLayers

# supply feature layer itemID, and the layer number to display table head 
def getLayerTable(gisInfo, idString, layerNum):
    try:
        layerOutput = getLayers(gisInfo, idString)
        queryLayer = layerOutput[layerNum].query()
        display(queryLayer.sdf.head())
    except:
        print('ERROR no Layers Found')
        
# delete item by itemID
def deleteItem(gisInfo, idString):
    itemToDelete = gisInfo.content.get(idString)
    display(itemToDelete)
    delQuest = input(f'Are you sure you want to delete: {itemToDelete.title}')
    try:
        if delQuest.upper() == 'YES':
            print(f'DELETING: {itemToDelete.title}')
            itemToDelete.delete()
        else:
            print(f'NOT DELETING: {itemToDelete.title}')
    except:
        print(f'ERROR failed to DELETE: {itemToDelete.title}')

# delete multiple items by searchByKeywords() returned LIST 8-14-2020
def delMultiple(gisInfo, itemList):
  try:
    print('List of Items to be Deleted: ')
    for z in itemList:
          strMod = str(z.modified)
          stampInt = int(strMod[0:10])
          print(f'{z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
    delQuest = input(f'SURE YOU WANT TO DELETE THESE?!?!?')
    if delQuest.upper() == 'YES':
      for z in itemList:
          print(f'DELETING {z.title}')
          itemToDelete = gisInfo.content.get(z.id)
          itemToDelete.delete()
      print('<FINISHED DELETION PROCESS>')
    else:
      print(f'NOT DELETING!')
  except:
      print(f'ERROR failed to DELETE: {itemToDelete.title}')

# List all user Dashboards and Dashboard Webmmaps
def ListAllDashWebmaps(gisInfo):
  source_admin_inventory = get_user_items(gisInfo, gisInfo.users.me)
  x = 0
  try:
    for dashboard in source_admin_inventory['Dashboard']:
        print(x, dashboard)
        dashWebmap = get_dash_wm(gisInfo, dashboard)
        print(dashWebmap)
        x += 1
  except:
    print("ERROR COULD NOT LIST DASHBOARDS") 

# generic function update targetLayer Features based on Table Records
def updateLayFeatFromTable(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      tableFeatureID = tableFeature.attributes[matchAttrib]
      for layerFeature in layerFeatures:
        layerFeatureID = layerFeature.attributes[matchAttrib]
        if tableFeatureID == layerFeatureID:
          targetValue = tableFeature.attributes[sourceAttrib]
          layerFeature.set_value(targetAttrib, targetValue)
          print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib} as: {targetValue}')
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
  except:
    print(f'update features failed for {getLayers.title}')
  
# generic function update targetLayer Features based on Table Record, adds break list for parameter categories mapping/analysis
# 8-14 Updated to screen for sampling records with blank values: 'None'
def updateLayFeatFromTableBreaks(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      #tableFeatureID = tableFeature.attributes[matchAttrib]
      print(tableFeature.attributes['WATER_TEMP'] is None)
      if (tableFeature.attributes['WATER_TEMP'] is None) != True:
        tableFeatureID = tableFeature.attributes[matchAttrib]
        for layerFeature in layerFeatures:
          layerFeatureID = layerFeature.attributes[matchAttrib]
          if tableFeatureID == layerFeatureID:
            targetValue = tableFeature.attributes[sourceAttrib]
            x = 1
            for breakVal in breaksList:
              if targetValue > breakVal:
                print('none')
              else:
                layerFeature.set_value(targetAttrib, x)
                print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib}: {targetValue} as: {x}')
                break
              x+=1
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    # hide REST infor for updated layers and tables
    # print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
    print(f'updated {getLayers.title} with {editCounter} edits')
  except:
    print(f'update features failed for {getLayers.title}')

# download Feature Layer data from AGOL, unzip contents to folder with item.title name
# Export Formats: Shapefile | CSV | File Geodatabase | Feature Collection | GeoJson | Scene Package | KML | Excel
def downloadItem(gisInfo, idString):
    try:
        downloadData = gisInfo.content.get(idString)
        dataPath = Path('/data')
        print(f'Downloading: {downloadData.title} to {dataPath} directory')
        if not dataPath.exists():
          dataPath.mkdir()
        # this portion for feature service
        downloadExport = downloadData.export(title=downloadData.title, export_format="CSV")
        zipPath = downloadExport.download(save_path=dataPath)
        # preparing to extract files to directory with item.title name
        #zipPath = downloadData.download(save_path=dataPath)
        extractPath = dataPath.joinpath(downloadData.title)
        # extract files to /data directory
        zipFiles = ZipFile(zipPath)
        zipFiles.extractall(path=extractPath)
        print(f'list of Files extracted to: {extractPath}')
        print(list(file.name for file in extractPath.glob('*')))
    except:
        print('ERROR DOWNLOAD did not workings!')

def searchItem(gisInfo, searchKeywords, itemType):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type=itemType, max_items=25)
        if itemType == 'Feature Service':
            x = 0
            print(f'<Search Query for {searchKeywords}>')
            for z in searchContent:
                strMod = str(z.modified)
                stampInt = int(strMod[0:10])
                print(f'{x} - {z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
                x += 1
            layInd = int(input(f'Index of selected Feature Layer: '))
            addLayer = gisInfo.content.get(searchContent[layInd].id)
            return addLayer
        elif itemType == 'Web Map':
            x = 0
            print(f'<Search Query for {searchKeywords}>')
            for z in searchContent:
                strMod = str(z.modified)
                stampInt = int(strMod[0:10])
                print(f'{x} - {z.title} - {z.id} - {z.type} - {z.categories} - {dt.datetime.fromtimestamp(stampInt)}')
                x += 1
            layInd = int(input(f'Index of selected Feature Layer: ')) or 'NONE'
            #print(searchContent[layInd])
            mapReturn = searchContent[layInd]
            return mapReturn
    except:
        print('ERROR Search not Successful')
        
def quickMap():
  mapType = input(f'(YES) for QuickMap (NO) for Existing: ')
  if mapType.upper() == 'NO':
    mapSize = ['480px','720px','960px']
    print(f'<You entered {mapType} please login below>')
    userLogin() 
    mapKeywords = input(f'Name of WebMap to Search for: ') or ''
    mapObj = searchItem(gis,mapKeywords,'Web Map')
    map = gis.map(mapObj)
    sizeIn = int(input(f'MAP SIZE (0)SMALL (1)MEDIUM (2)HUGE: '))
    map.layout.height = mapSize[sizeIn]
    display(map)
  else:
    print(f'<You entered {mapType} Opening QuickMap>')
    mapList = ['topo','hybrid','streets','dark-gray','terrain']
    mapDimen = ['2D','3D']
    mapSize = ['480px','720px','960px']
    mapLoc = input(f'Location (default=Michigan): ') or 'Michigan'
    mapBaseNum = input(f'Basemap (default=topo (1=hybrid,2=streets,3=dark-gray,4=terrain): ') or 0
    mapDimIn = input(f'ENTER (1) for 3D Map: ') or 0
    atlasLayers = input(f'Layers from Living Atlas(Enter for None): ') or 'NONE'
    gisNone = GIS()
    map = gisNone.map(mapLoc)
    map.basemap = mapList[int(mapBaseNum)]
    if atlasLayers != 'NONE':
        layerDisplay = searchItem(gisNone, atlasLayers,'Feature Service')
        for layrs in layerDisplay.layers:
            map.add_layer(layrs)
    map.mode = mapDimen[int(mapDimIn)]
    sizeIn = int(input(f'MAP SIZE (0)SMALL (1)MEDIUM (2)HUGE: '))
    map.layout.height = mapSize[sizeIn]
    display(map)

# Delete all features from selected Feature Service, may need more debugging
def delAllFeatures(gisInfo, idString):
    delFeatures = gisInfo.content.get(idString)
    display(delFeatures)
    delQuest = input(f'Are you sure you want to delete all the feature in? {delFeatures.title}')
    try:
        if delQuest.upper() == 'YES':
            print('in loop')
            print(f'DELETING ALL FEATURES IN: {delFeatures.title}')
            featDelRes = []
            targetLayer = delFeatures.layers
            layerFeatures = targetLayer[0].query()
            for feature in layerFeatures:
              #print(f'features: {feature.attributes}')
              featDelRes.append(targetLayer[0].edit_features(deletes=str(feature.attributes['objectid'])))
            return featDelRes
        else:
            print(f'NOT DELETING: {delFeatures.title}')
    except:
        print(f'ERROR failed to DELETE: {delFeatures.title}')

# *******ESRI pre-made helper functions********
def is_hosted(gisInfo, item):
    return [keyword for keyword in item.typeKeywords if "Hosted" in keyword] 

# Prints all layers in a webmap, very handy
def print_webmap_inventory(gisInfo, wm):
    wm_obj = WebMap(wm)
    print(f"{wm_obj.item.title}\n{'-'*100}")
    for wm_layer in wm_obj.layers:
        try:
            if is_hosted(Item(gisInfo, wm_layer['itemId'])):
                print(f"{' '*2}{wm_layer['title']:40}HOSTED{' ':5}"
                      f"{wm_layer['layerType']:20}{dict(wm_layer)['itemId']}")
            else:
                print(f"{' '*2}{wm_layer['title']:40}other{' ':6}"
                      f"{wm_layer['layerType']:20}{wm_layer.id}") 
        except:
            print(f"{' '*2}{wm_layer['title']:40}other{' ':6}"
                  f"{wm_layer['layerType']:20}{wm_layer.id}")
    print("\n")

def get_webmap_list(wm):
    wm_obj = WebMap(wm)
    wmList = []
    print(f"{wm_obj.item.title}\n{'-'*100}")
    for wm_layer in wm_obj.layers:
        # print(wm_layer.itemId)
        wmList.append(wm_layer.itemId)
    return(wmList)
    
def displayWebmapLayers(gisInfo, idList):
    for id in idList:
        displayLayer = gisInfo.content.get(id)
        display(displayLayer)

def get_user_items(gisInfo, user):
    user_inventory = {}
    user_items = gisInfo.content.search(query=f"* AND owner:{user.username}", 
                                           max_items=500)
    for item in user_items:
        if item.type not in user_inventory:
            user_inventory[item.type] = [i 
                                         for i in user_items 
                                         if i.type == item.type]
    return user_inventory

def print_user_inventory(inventory):
    for itype, ilist in inventory.items():
        try:
            print(f"{itype}\n{'-'*50}")
            for i in ilist:
                print(f"{' ':3}{i.title:50}")
            print("\n")
        except Exception as e:
            print(f"\t\tOperation failed on: {i.title}")
            print(f"\t\tException: {sys.exc_info()[1]}")
            continue
            
def get_dash_wm(gisInfo, dash):
    return [gisInfo.content.get(widget['itemId']) 
            for widget in dash.get_data()['widgets'] 
            if widget['type'] == "mapWidget"]

# Python API Function List  
**userLogin()** - **additionalUserLogin()** - **getUserContent** *(gisInfo)* - **cloneItem** *(gisInfo, gisInfo2, cloneID)*  
**searchByKeywords** *(gisInfo, searchKeywords)* - **searchByKeyViz** *(gisInfo, searchKeywords)*  
**getItemKeysValues** *(gisInfo, idString)* - **getLayers** *(gisInfo, idString)*
**getLayerTable** *(gisInfo, idString, layerNum)* - **deleteItem** *(gisInfo, idString)*  
**ListAllDashWebmaps** *(gisInfo)* - **updateLayFeatFromTable** *(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib)*  
**updateLayFeatFromTableBreaks** *(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList)*  
**downloadItem** *(gisInfo, idString)* - **searchItem** *(gisInfo, searchKeywords, itemType)* - **quickMap()**   
**delAllFeatures** *(gisInfo, itemID)* - **delMultiple** *(gisInfo, itemList)*


---


# ESRI pre-made helper functions  
**is_hosted** *(gisInfo, item)* - **print_webmap_inventory** *(gisInfo, wm)* - **get_webmap_list** *(wm)*  
**displayWebmapLayers** *(gisInfo, idList)* - **get_user_items** *(gisInfo, user)*  
**print_user_inventory** *(inventory)* - **get_dash_wm** *(gisInfo, dash)*


# Work in Progress Cell(s)

In [ ]:
#userLogin()


In [ ]:
#searchInput = 'kzoo'
#searchByKeywords(gis, searchInput)
searchInfo = ''
searchType = 'Feature Service'
searchReturn = searchItem(gis, searchInfo, searchType)
print(searchReturn.id)

# Working to show readable date when item was updated
Trying for format ESRI UTC date into standard day-month-year: time

In [ ]:
# find item by keywords and display text information
# recently added proper parsing of dates
def searchByKeywords(gisInfo, searchKeywords):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type='', max_items=50)
        for z in searchContent:
          strMod = str(z.modified)
          stampInt = int(strMod[0:10])
          print(f'title: {z.title} - itemID: {z.id} - type: {z.type}  - dateEdited: {dt.datetime.fromtimestamp(stampInt)}')
    except:
        print('ERROR Search not Successful')

searchInput = 'water data 2020'
searchByKeywords(gis, searchInput)

In [ ]:
)x = dt.datetime.now()
print(x)
formattedTime = x.strftime("%m/%d/%Y %X")
print(formattedTime)
#print(timeRaw)
testTime = time.gmtime(1000)
print(testTime)
#print(x.year)
#print(x.strftime("%A"))
#datetimeVal = 
 #   formattedTime = datetime.strftime(datetimeVal, "%m/%d/%Y")

2020-07-29 19:35:59.579742
07/29/2020 07:35:59 PM
time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=16, tm_sec=40, tm_wday=3, tm_yday=1, tm_isdst=0)


In [ ]:
tm.time()
dt.datetime.fromtimestamp(tm.time())

datetime.datetime(2020, 7, 29, 19, 37, 46, 195185)

In [ ]:
import math
x = -0
print(math.isnan(x))

False


# Water Quality Dashboard Display Update Functions
Updated KZOO Turb Status 7-23-2020 3:05 pm   
Updated NHBP DO Status 7-23-2020 3:08 pm
Upated NHBP DO Status 8-12-2020    
Updated KZOO Turb Status 8-15-2020 - Refactored Update Layer function, check for 'None'

In [ ]:
# generic function update targetLayer Features based on Table Record, adds break list for parameter categories mapping/analysis
# 8-14 Updated to screen for sampling records with blank values: 'None'
def updateLayFeatFromTableBreaks(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      #tableFeatureID = tableFeature.attributes[matchAttrib]
      print(tableFeature.attributes['WATER_TEMP'] is None)
      if (tableFeature.attributes['WATER_TEMP'] is None) != True:
        tableFeatureID = tableFeature.attributes[matchAttrib]
        for layerFeature in layerFeatures:
          layerFeatureID = layerFeature.attributes[matchAttrib]
          if tableFeatureID == layerFeatureID:
            targetValue = tableFeature.attributes[sourceAttrib]
            x = 1
            for breakVal in breaksList:
              if targetValue > breakVal:
                print('none')
              else:
                layerFeature.set_value(targetAttrib, x)
                print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib}: {targetValue} as: {x}')
                break
              x+=1
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    # hide REST infor for updated layers and tables
    # print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
    print(f'updated {getLayers.title} with {editCounter} edits')
  except:
    print(f'update features failed for {getLayers.title}')

#userLogin()
kzooLayerNHBP = 'dff379381a6b4b73a1d80b9fd42784a8'
kzooLayerLA = '6b62d8b710e64b8abc79015fd7231b87'
waterSampNHBP = '680016d676e746f98743f51d28abac60'
match = 'SITE_ID'
turbTarget = 'TURB_STATUS_'
turbSource = 'TURB_NTU'
turbRefList = [3.93,10,40,1000]
DOtarget = 'DO_STATUS_'
DOsource = 'DO_mgl'
DOrefList = [5,6,7,100]
# calc Turbidity Status from most recent sample
#updateLayFeatFromTableBreaks(gis, kzooLayerNHBP, match, turbTarget, turbSource, turbRefList)
# calc DO Status from most recent sample
updateLayFeatFromTableBreaks(gis, waterSampNHBP, match, DOtarget, DOsource, DOrefList)

# **Working function to download and process WATER DATA**

1.   (COMP) Download and extract WATER_SAMPLING_2020 (itemID: '680016d676e746f98743f51d28abac60')  
2.   (COMP) Read Water Data Sample Table as Pandas Dataframe  
3. (COMP)Drop uneeded columns/attributes  
4. (NEXT STEP) Format correctly for AWQMS upload
5. Write as XLS for export to AWQMS


In [ ]:
# downloadReturn and exportAGOLdata work together with specified itemID to 
# package Export from AGOL into CSV, download and extract files, pass file
# path back to export func which creates pandas dataframe, slices off desired 
# attributes/columns and prints the dataframe head.
# NOTE: paths seem to switch the CSV order, one is features, one sample records
# Added ExportDataFrame 8-25, works great!
def downloadReturn(gisInfo, idString):
    try:
        downloadData = gisInfo.content.get(idString)
        dataPath = Path('/data')
        print(f'Downloading: {downloadData.title} to {dataPath} directory')
        if not dataPath.exists():
          dataPath.mkdir()
        # this portion for feature service
        downloadExport = downloadData.export(title=downloadData.title, export_format="CSV")
        zipPath = downloadExport.download(save_path=dataPath)
        # preparing to extract files to directory with item.title name
        extractPath = dataPath.joinpath(downloadData.title)
        # extract files to /data directory
        zipFiles = ZipFile(zipPath)
        zipFiles.extractall(path=extractPath)
        # delete CSV created by export
        print(f'Deleting CSV export generated by Download: {downloadExport.title} ID: {downloadExport.id}')
        itemToDelete = gisInfo.content.get(downloadExport.id)
        itemToDelete.delete()
        print(f'list of Files extracted to: {extractPath}')
        print(list(file.name for file in extractPath.glob('*')))
        returnArray = []
        for x in extractPath.glob('*'):
          returnArray.append(x)
        return returnArray
    except:
        print('ERROR DOWNLOAD did not workings!')

def exportDataFrame(df,colRename,exportName):
  dfRN = df.rename(colRename, axis='columns')
  # Very specific edits to generate AWQMS outputs
  #dfRN['FIELD Activity ID'] = dfRN['TURB']
  # End of VSA
  writer = pd.ExcelWriter(exportName, engine='xlsxwriter')
  dfRN.to_excel(writer, sheet_name='Sheet1', index=False)
  writer.save()
  print(f'dataframe successfully exported as: {exportName}')
  return dfRN

# function to download and process data as specified above
def exportAGOLdata(gis, itemID, exportName='exportXLS.xlsx', sliceArray=['BLANK'],colNames={}):
  try:
    DLdata = downloadReturn(gis, itemID)
    if str(DLdata[0])[-5] == '1':
      fullPath = DLdata[0]
    else: 
      fullPath = DLdata[1]
    print(fullPath)
    df = pd.read_csv(fullPath)
    display(df.head())
    if sliceArray[0] != 'BLANK':
      columns = df.columns
      columnSlice = columns[sliceArray[0]:sliceArray[1]]
      df = df.drop(columnSlice,axis=1)
    dfED = exportDataFrame(df,colNames,exportName)
    print('<Dataframe Header created from AGOL item Export>')
    display(dfED.head().T)
    return dfED
  except:
    print(f'ERROR with {itemID} EXPORT')



#userLogin()
searchReturn = searchByKeywords(gis, 'water data 2020 Feature Service')
indexNumber = input(f'What is the index of the file search target? ')
dfEX = exportAGOLdata(gis,searchReturn[int(indexNumber)].id,'xportXLS.xlsx')
dfExport = formatXLS(dfEX)
dfExport

In [ ]:
# final formatting functions for XLS export

#userLogin()
searchReturn = searchByKeywords(gis, 'water data 2020 Feature Service')
indexNumber = input(f'What is the index of the file search target? ')
dfEX = exportAGOLdata(gis,searchReturn[int(indexNumber)].id,'xportXLS.xlsx')
dfExport = formatXLS(dfEX)
dfExport

In [ ]:
# Easily creates XLS from dataframe, working 8-25
writer = pd.ExcelWriter('demo.xlsx', engine='xlsxwriter')
dfEX.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()

In [ ]:
dfEX.head()

In [ ]:
#This is SOOOOOO CLOSE!!!
#All fields and formatting looks great, just need to add QC conditional tag to the end of Activity ID Field and done!
#dfEX
#Done 9-8-2020 - Just need to organize!
def formatXLS(dfEX):

  def actType(QCval):
      if QCval == 'Field Measurement':
        return 'Field Msr/Obs'
      else:
        return 'Quality Control Field Replicate Msr/Obs'

  def FtoC(Ftemp):
    Ctemp = (int(Ftemp) - 32) * (5/9)
    return round(Ctemp, 2)

  def getDate(dateTime):
    splitDate = dateTime.split('/')
    #print(f'splitDate: {splitDate}')
    if int(splitDate[0]) < 10:
      monthName = '0' + splitDate[0]
    else:
      monthName = splitDate[0]
    if int(splitDate[1]) < 10:
      dayName = '0' + splitDate[1]
    else:
      dayName = splitDate[1]
    splitDate = splitDate[2].split(' ')
    sampYear = splitDate[0]
    dateString = (f'{sampYear}{monthName}{dayName}')
    return dateString

  def getTime(dateTime):
    dateSplit = dateTime.split(' ')
    timeSplit = dateSplit[1].split(':')
    getHours = int(timeSplit[0]) + 7
    sampTime24 = str(getHours) + timeSplit[1]
    return sampTime24

  def isQC(qcInfo):
    returnVal = ''
    if qcInfo == 'Quality Control Sample Field Replicate':
      returnVal = ':QC'
    return returnVal

  siteList = {'DKC-ST-30':1,'DKC-ST-50':2,'FDP-SD-10':3,'ICD-ST-20':4,'ICD-ST-40':5,'ICD-ST-60':6,'KAR-ST-100':7,'KAR-ST-200':8,'NOT-ST-110':9,'NOT-ST-120':10,'NOT-ST-130':11,'NOT-ST-30':12,'NOT-ST-50':13,'NOT-ST-60':14,'NOT-ST-70':15,'NOT-ST-80':16,'NOT-ST-90':17,'PGC-ST-30':18,'PNC-ST-20':19,'PNC-ST-30':20,'PNC-ST-40':21,'PNC-ST-50':22,'PNC-ST-60':23,'PNC-ST-70':24,'QDP-LA-10':25,'RDP-SD-10':26,'SCD-ST-20':27,'SCD-ST-40':28,'SCD-ST-50':29,'SDP-LA-10':30,'UNT-ST-10':31,'UNT-ST-20':32,'ICD-ST-70':33,'HVC-ST-20':34,'NOT-ST-140':35,'NOT-ST-40':36,'DKC-ST-20':37,'DKC-ST-25':38,'NOT-ST-115':39,'PNC-ST-45':40,'ICD-ST-65':41,'SPC-ST-90':42,'SJR-ST-300':43}

  dfV2 = pd.DataFrame()
  dfEX['SITE #'] = dfEX.apply(lambda row: siteList[row.SITE_ID], axis=1)
  dfV2['SITE #'] = dfEX.apply(lambda row: siteList[row.SITE_ID], axis=1)
  numString = 'SITE #'
  dfV2['FIELD Activity ID'] = dfEX.apply(lambda row: row.SITE_ID + ':' + str(row['SITE #']) + ':'+ getDate(row.DATE_TIME)+':'+ getTime(row.DATE_TIME) + isQC(row.QC), axis = 1)
  dfV2['Activity Type'] = dfEX.apply(lambda x: actType(x['QC']), axis = 1)
  dfV2['DATE'] = dfEX.apply(lambda row: row.DATE_TIME[0:8], axis = 1)
  dfV2['TIME'] = dfEX.apply(lambda row: row.DATE_TIME[9:19], axis = 1)
  dfV2['TEMP C'] = dfEX.apply(lambda x: FtoC(x['WATER_TEMP']), axis = 1)
  dfV2['SPCOND'] = dfEX['SPEC_COND_uS_cm']*.001
  dfV2['DO%'] = ''
  dfV2['pH'] = round(dfEX['pH'],1)
  dfCol = ['Monitoring Location','Sample Collection Equipment Name','TEMP F','TURB','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']
  srcCol = ['SITE_ID','SAMP_EQUIP','WATER_TEMP','TURB_NTU','DO_mgl','TOT_NITRO_','TOT_PHOS_','Ecoli_100ml_','NOTES']

  x = 0
  for col in dfCol:
    dfV2[col] = dfEX[srcCol[x]]
    x+=1

  dfV2 = dfV2[['FIELD Activity ID','Monitoring Location','SITE #','Activity Type','Sample Collection Equipment Name','DATE','TIME','TEMP C','TEMP F','SPCOND','pH','TURB','DO%','DO mg/L','TOT. N mg/L','TOT. P mg/L','E. coli col/100 ml','NOTES']]
  
  return dfV2
#dfV2

In [ ]:
siteList = {'DKC-ST-30':1,'DKC-ST-50':2,'FDP-SD-10':3,'ICD-ST-20':4,'ICD-ST-40':5,'ICD-ST-60':6,'KAR-ST-100':7,'KAR-ST-200':8,'NOT-ST-110':9,'NOT-ST-120':10,'NOT-ST-130':11,'NOT-ST-30':12,'NOT-ST-50':13,'NOT-ST-60':14,'NOT-ST-70':15,'NOT-ST-80':16,'NOT-ST-90':17,'PGC-ST-30':18,'PNC-ST-20':19,'PNC-ST-30':20,'PNC-ST-40':21,'PNC-ST-50':22,'PNC-ST-60':23,'PNC-ST-70':24,'QDP-LA-10':25,'RDP-SD-10':26,'SCD-ST-20':27,'SCD-ST-40':28,'SCD-ST-50':29,'SDP-LA-10':30,'UNT-ST-10':31,'UNT-ST-20':32,'ICD-ST-70':33,'HVC-ST-20':34,'NOT-ST-140':35,'NOT-ST-40':36,'DKC-ST-20':37,'DKC-ST-25':38,'NOT-ST-115':39,'PNC-ST-45':40,'ICD-ST-65':41}
siteList['HVC-ST-20']
siteList[dfEX['SITE_ID'][0]]

5

# **Explore Excel in Pandas**

In [ ]:
searchByKeywords(gis, 'water data 2020')

In [ ]:
downloadItem(gis, '680016d676e746f98743f51d28abac60')

Downloading: WATER_SAMPLING_2020 to /data directory
list of Files extracted to: /data/WATER_SAMPLING_2020
['WATER_STATIONS_2020_0.csv', 'WATER_DATA_2020_1.csv']


In [ ]:
#import pandas as pd
#waterDF = pd.read_csv('/data/WATER_SAMPLING_2020/WATER_DATA_2020_1.csv')
#waterDF.head()
#waterDF.info()
#waterDF.describe()
#waterDF[0:5]['SITE_ID']
#siteID = waterDF.loc[0]['SITE_ID']
#siteID
#waterDF.iloc[0]
#columns = waterDF.columns
#x=0
#for col in columns:
  #print(x, col)
  #x+=1
#columns[0:18]
#waterDF = waterDF.drop(,axis=1)
waterDF.head()
writer = pd.ExcelWriter('demo.xlsx', engine='xlsxwriter')
writer.save()

colSlice = columns[18:31]
#for col in colSlice:
  #print(col)
waterDF = waterDF.drop(colSlice,axis=1)

In [ ]:
colSlice = columns[18:31]
#for col in colSlice:
  #print(col)
waterDF = waterDF.drop(colSlice,axis=1)

In [ ]:
writer = pd.ExcelWriter('demo.xlsx', engine='xlsxwriter')
writer.save()

In [ ]:
heads = ['SITE_ID','WATERBODY']
datas = [['PNC-ST-50','NOT-ST-70'],['PINE CREEK','NOTTAWA CREEK']]
df = pd.DataFrame(frameData)
frameData = {heads[0]:datas[0],heads[1]:datas[1]}
df = pd.DataFrame(frameData)
writer = pd.ExcelWriter('demo.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()

In [ ]:
df2 = pd.read_excel('demo.xlsx', index_col=None, no_values=['NA'])
df2.head()

In [ ]:
# new dataframe with same columns
df = pd.DataFrame({'Nsdsdsdadada': ['E','F','G','H'],
                   'Age': [100,70,40,60], 'param3': [120,1383,293,283]})
writer = pd.ExcelWriter('demo.xlsx', engine='openpyxl')
# try to open an existing workbook
writer.book = load_workbook('demo.xlsx')
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
# read existing file
reader = pd.read_excel(r'demo.xlsx')
# write out the new sheet
df.to_excel(writer,index=False,header=False,startrow=len(reader)+1)
writer.close()
df2 = pd.read_excel('demo.xlsx', index_col=None, no_values=['NA'])
df2

# **Pandas Exploration of Titantic Dataset**

In [ ]:
import pandas as pd

In [ ]:
titanticDF = pd.read_excel('TitanicTrain.xlsx', index_col=None, no_values=['NA'])

In [ ]:
titanticDF.head()

In [ ]:
titanticDF.describe()

In [ ]:
#titanticDF.drop()
pd.value_counts(titanticDF['Survived']).plot.bar()

In [ ]:
titanticDF['Survived'].mean()

0.3838383838383838

In [ ]:
titanticDF.groupby(['Sex']).mean()

In [ ]:
titanticDF.groupby(['Sex','Pclass']).mean()

In [ ]:
titanticDF[titanticDF['Age']<18].groupby(['Sex','Pclass']).mean()